In [3]:
import pandas as pd
import sqlite3

In [16]:
db = sqlite3.connect('../../datasets/checking-logs.sqlite')

## Scheme

In [5]:
pd.read_sql('PRAGMA table_info(test);',db)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## Test first 10

In [6]:
pd.read_sql('SELECT * FROM test LIMIT 10', db)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [7]:
pd.read_sql(""" SELECT * FROM deadlines """, db)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


## Minimum delta

In [8]:
df_min = pd.read_sql(""" SELECT t.uid, MIN((strftime('%s', t.first_commit_ts) - d.deadlines) /3600) AS min_delta
                     FROM test t
                     JOIN deadlines d ON t.labname = d.labs
                     WHERE t.labname != 'project1' """, db)
df_min

,uid,min_delta
0,user_30,-202


## Maximum delta

In [9]:
df_max = pd.read_sql(""" SELECT t.uid, MAX((strftime('%s', t.first_commit_ts) - d.deadlines) /3600) AS max_delta
                     FROM test t
                     JOIN deadlines d ON t.labname = d.labs
                     WHERE t.labname != 'project1' """, db)
df_max

,uid,max_delta
0,user_25,-2


## AVG

In [10]:
df_avg = pd.read_sql(""" SELECT AVG((strftime('%s', t.first_commit_ts) - d.deadlines) /3600) AS avg
                     FROM test t
                     JOIN deadlines d ON t.labname = d.labs
                     WHERE t.labname != 'project1' """, db)
df_avg

,avg
0,-89.125


## Tst hip

In [11]:
pd.read_sql(""" SELECT * FROM pageviews""", db)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
...,...,...,...
1075,1075,user_25,2020-05-21 23:23:49.995349
1076,1076,admin_1,2020-05-21 23:49:22.386789
1077,1077,admin_1,2020-05-22 10:36:14.662600
1078,1078,user_5,2020-05-22 11:30:18.368990


In [20]:
views_diff = pd.read_sql(""" SELECT t.uid, AVG((strftime('%s', t.first_commit_ts) - d.deadlines) /3600) AS avg_diff, 
                  COUNT(p.uid) AS pageviews
                  FROM test t
                  JOIN deadlines d ON t.labname = d.labs
                  LEFT JOIN pageviews p ON t.uid = p.uid
                  WHERE t.labname != 'project1'
                  GROUP BY t.uid """, db)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,140
1,user_10,-74.800000,445
2,user_14,-159.000000,429
3,user_17,-61.600000,235
4,user_18,-5.666667,9
5,user_19,-98.750000,64
6,user_21,-95.500000,40
7,user_25,-92.600000,895
8,user_28,-86.400000,745
9,user_3,-105.400000,1585


In [21]:
print(views_diff[[ 'avg_diff', 'pageviews',]].corr())

           avg_diff  pageviews
avg_diff   1.000000  -0.185834
pageviews -0.185834   1.000000


In [22]:
db.close()